# Projeto 2: Como pegar dados em sites e automatizar a criação da carteira do nosso modelo.

### Desafio:

* Construir um código que vá no fundamentus.com e busque dados de todos os indicadores da bolsa brasileira e gere a carteira selecionando as maiores ev_ebit e roic.    


### Passo a passo:

   **Passo 1** - Definir um navegador que você irá utilizar para navegar com o Python.

   **Passo 2** - Importar os módulos e bibliotecas.
   
   **Passo 3** - Entender como funcionam requisições na internet.
   
   **Passo 4** - Entender como sites funcionam.
   
   **Passo 5** - Conhecer e mapear o processo de coleta de dados no site do Fundamentus. 
   
   **Passo 6** - Ler a tabela de dados.
   
   **Passo 7** - Construir a tabela final.

# Passo 1: Escolher o navegador.

No nosso caso, utilizaremos o Google Chrome. 

# Passo 2: Importar as bibliotecas.

In [ ]:
!pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Passo 3: Entender como funcionam requisições na internet.

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://www.fundamentus.com.br/resultado.php'

driver.get(url)

# Passo 4: Entender como sites funcionam.

# Passo 5: Mapear o processo de coleta de dados no site do Fundamentus.

https://www.fundamentus.com.br/index.php

# Passo 6 - Ler a tabela de dados.

In [ ]:
local_tabela = '/html/body/div[1]/div[2]/table'

elemento = driver.find_element("xpath", local_tabela)

html_tabela = elemento.get_attribute('outerHTML')

tabela = pd.read_html(str(html_tabela), thousands = '.', decimal = ',')[0]

# Passo 7 - Construir a tabela final.

In [ ]:
tabela = tabela.set_index("Papel")
tabela = tabela[['Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses']]

tabela

### Temos que transformar o texto em números..

In [ ]:
tabela.info()

#tabela['EV/EBIT'] * tabela['ROIC']

# Passo 7.2: Tratar o texto.

In [ ]:
tabela['ROIC'] = tabela['ROIC'].str.replace("%", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(".", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(",", ".")
tabela['ROIC'] = tabela['ROIC'].astype(float)
display(tabela)

# Passo 7.3: Fazendo os filtros e criando os rankings. 

In [ ]:
tabela = tabela[tabela['Liq.2meses'] > 1000000]

In [ ]:
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]

In [ ]:
tabela['ranking_ev_ebit'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['ranking_roic'] = tabela['ROIC'].rank(ascending = False)
tabela['ranking_total'] = tabela['ranking_ev_ebit'] + tabela['ranking_roic']

tabela = tabela.sort_values('ranking_total')

tabela.head(10)